## ДЗ по поиску

Привет! Вам надо реализивать поисковик на базе вопросов-ответов с сайта [pravoved.ru](https://pravoved.ru/questions-archive/).        
Поиск должен работать на трех технологиях:       
1. обратном индексе     
2. word2vec         
3. doc2vec      

Вы должны понять, какой метод и при каких условиях эксперимента на этом корпусе работает лучше.          
Для измерения качества поиска найдите точность (accuracy) выпадания правильного ответа на конкретный вопрос (в этой базе у каждого вопроса есть только один правильный ответ). Точность нужно измерить для всей базы.    
При этом давайте считать, что выпал правильный ответ, если он попал в **топ-5** поисковой выдачи.

> Сделайте ваш поиск максимально качественным, чтобы значение точности стремилось к 1.     
Для этого можно поэкспериментировать со следующим:       
- модель word2vec (можно брать любую из опен сорса или обучить свою)
- способ получения вектора документа через word2vec: простое среднее арифметическое или взвешивать каждый вектор в соответствии с его tf-idf      
- количество эпох у doc2vec (начинайте от 100)
- предобработка документов для обучения doc2vec (удалять / не удалять стоп-слова)
- блендинг методов поиска: соединить результаты обратного индекса и w2v, или (что проще) w2v и d2v

На это задание отведем 10 дней. Дэдлайн сдачи до полуночи 12.10.

In [1]:
import pickle

with open('qa_corpus.pkl', 'rb') as file:
    qa_corpus = pickle.load(file)

In [2]:
set_ = (1, 2, 3, 4)
set_ = set(set_)
len(set_)

4

Всего в корпусе 1384 пары вопрос-ответ

In [3]:
len(qa_corpus)

1384

Первый элемент блока это вопрос, второй - ответ на него

In [4]:
qa_corpus[0][1]

'Добрый вечер!Из Вашего вопроса вообще ничего не ясно.Ваш сын по ВНЖ в Нижегородской обл. сделал временную\xa0 на 90 дней в Ростове? Так? Или в чем заключается вопрос?С ув., АлёнаМиграционный юристРостов-на-Дону '

In [5]:
questions = []
answers = []
for corp in qa_corpus:
    questions.append(corp[0])
    answers.append(corp[1])

In [6]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
from gensim.models import Word2Vec, KeyedVectors
from stop_words import get_stop_words
import string
from pymystem3 import Mystem
from nltk.tokenize import word_tokenize
mystem = Mystem()
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
import json
import numpy as np

In [7]:
model_path = 'D:/tayga_1_2.vec'
model = KeyedVectors.load_word2vec_format(model_path, binary=False)

In [8]:
from gensim import matutils


def similarity(v1, v2):
    v1_norm = matutils.unitvec(np.array(v1))
    v2_norm = matutils.unitvec(np.array(v2))
    return np.dot(v1_norm, v2_norm)

In [27]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. normal form
        3. del stopwords
        4. del digits
    :return: lemmas
    """
    russian_stopwords = set(get_stop_words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
#     lemmas = morph.parse(x)[0].normal_form

    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        pos = morph.parse(lemma)[0].tag.POS
        lemma = lemma + '_' + str(pos)
        lemmas_arr.append(lemma)
#     lemmas_ = ' '.join(lemmas_arr)
    return lemmas_arr

def get_w2v_vectors(lemmas_arr, text):
    """Получает вектор документа"""
    dict_ = {}
    feature_vec = np.zeros((300, ), dtype='float32')
    
    for lemma in lemmas_arr:
        try:
#             my_vec.append(model.wv[lemma].tolist())
            feature_vec = np.add(feature_vec, model.wv[lemma])
#             print(len(model.wv[lemma]))
        except KeyError:
            print(lemma)
            feature_vec = np.add(feature_vec, np.zeros((300, ), dtype='float32'))
    text = text.strip()
    dict_ = {text :  feature_vec}
#     print(dict_)
#     print(dict_)
#     json_ = json.dumps(dict_, ensure_ascii=False)
    return dict_
  

def save_w2v_base(dict_, base_dict):
    """Индексирует всю базу для поиска через word2vec"""

    base_dict.update(dict_)
#     print(base_dict)
#     base_json = json.dumps(base_dict, ensure_ascii=False)
    return base_dict

In [10]:
base_dict = {}
for answer in answers:
    prepr = preprocessing(answer, del_stopwords=True, del_digit=True)
    get_wv = get_w2v_vectors(prepr, answer)
    save_wv = save_w2v_base(get_wv, base_dict)

c:\users\maria tsareva\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:37: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


добрый_ADJF
вообще_ADVB
ничто_NPRO
ясно.ваш_NOUN
внж_None
нижегородский_ADJF
сделать_INFN
временной_ADJF
заключаться_INFN
алёнамиграционный_ADJF
юристростов-на-дону_NOUN
вправе_PRED
платить_INFN
являться_INFN
​_None
недвижимый_ADJF
получить_INFN
физический_ADJF
признавать_INFN
семейный_ADJF
российский_ADJF
—_None
недвижимый_ADJF
получить_INFN
—_None
недвижимый_ADJF
получить_INFN
—_None
пожизненный_ADJF
иждивением.ить_INFN
—_None
случаях.этый_ADJF
установить_INFN
пп_None
217.1_None
нк_None
здравствовать_INFN
подать_INFN
административный_ADJF
копия-скан_NOUN
т.д_None
электронный_ADJF
например_CONJ
разместить_INFN
флешек_NOUN
dvd-диск_NOUN
возможно_CONJ
гпк_NOUN
рф.гпк_NOUN
участвовать_INFN
1.1_None
участвовать_INFN
вправе_PRED
представлять_INFN
бумажный_ADJF
электронный_ADJF
электронный_ADJF
подписать_INFN
электронный_ADJF
установленный_ADJF
российский_ADJF
заполнять_INFN
разместить_INFN
официальный_ADJF
информационно-телекоммуникационный_ADJF
1.2_None
участвовать_INFN
вправе_PRED
предст

здравствовать_INFN
елена.безусловный_ADJF
вначале_ADVB
правда_PRCL
необходимый_ADJF
отправить_INFN
изложить_INFN
свой_ADJF
выполнить_INFN
приложить_INFN
составить_INFN
перечислить_INFN
подробно_ADVB
направить_INFN
поступить_INFN
обращаться_INFN
приложить_INFN
постовой_ADJF
добрый_ADJF
​ск_None
родительский_ADJF
проживать_INFN
отдельно_ADVB
проживать_INFN
отдельно_ADVB
проживать_INFN
должный_ADJF
препятствовать_INFN
причинять_INFN
физический_ADJF
психический_ADJF
нравственный_ADJF
развитию.​таким_NOUN
препятствовать_INFN
смочь_INFN
доказать_INFN
указанный_ADJF
наносить_INFN
физический_ADJF
нравственный_ADJF
всякий_ADJF
участвовать_INFN
подобный_ADJF
судебный_ADJF
дать_INFN
исходить_INFN
принимать_INFN
никакой_ADJF
подумать_INFN
родительский_ADJF
p.s_None
детальный_ADJF
интересовать_INFN
расширить_INFN
исковый_ADJF
иной_ADJF
обратиться_INFN
—_None
здравствовать_INFN
​_None
владимирский_ADJF
областина_NOUN
несовершеннолетний_ADJF
некоторый_ADJF
алкоголя.статие_NOUN
установить_INFN
владими

добрый_ADJF
планировать_INFN
вести_INFN
предпринимательский_ADJF
необходимый_ADJF
оформить_INFN
соответствующий_ADJF
правовой_ADJF
ооо_NOUN
ип​_None
противный_ADJF
попехать_INFN
коап_NOUN
14.1_None
предпринимательский_ADJF
государственный_ADJF
специальный_ADJF
предпринимательский_ADJF
государственный_ADJF
индивидуальный_ADJF
государственный_ADJF
юридический_ADJF
предусмотренный_ADJF
частью_ADVB
14.17.1_None
настоящий_ADJF
—_None
влечь_INFN
административный_ADJF
пятьсот_NUMR
рублей.​с_NOUN
p.s_None
детальный_ADJF
интересовать_INFN
расширить_INFN
исковый_ADJF
иной_ADJF
обратиться_INFN
—_None
юристу.​_None
здравствовать_INFN
смочь_INFN
кто-либо_NPRO
заблокировать_INFN
снять_INFN
деньгимаринануть_INFN
иной_ADJF
судебный_ADJF
наложить_INFN
пристав_GRND
незаконный_ADJF
применяться_INFN
—_None
фз_None
законный_ADJF
физический_ADJF
просрочить_INFN
федеральный_ADJF
микрофинансовый_ADJF
микрофинансовый_ADJF
действующий_ADJF
обязать_INFN
возместить_INFN
компенсировать_INFN
моральный_ADJF
причинит

​ответственность_NOUN
установить_INFN
российский_ADJF
административный_ADJF
30.12.2001_None
n_None
195-фз_NOUN
ред_None
20.12.2017_None
изм_None
доп_None
вступа_NOUN
31.12.2017_None
коап_NOUN
5.27_None
трудовой_ADJF
иной_ADJF
нормативный_ADJF
правовой_ADJF
содержимый_ADJF
трудовой_ADJF
ред_None
федеральный_ADJF
03.07.2016_None
n_None
272-фз_NOUN
предыдущий_ADJF
трудовой_ADJF
иной_ADJF
нормативный_ADJF
правовой_ADJF
содержимый_ADJF
трудовой_ADJF
иной_ADJF
предусмотреть_INFN
настоящий_ADJF
5.27.1_None
настоящий_ADJF
—_None
влечь_INFN
административный_ADJF
должностной_ADJF
осуществлять_INFN
предпринимательский_ADJF
юридический_ADJF
—_None
юридический_ADJF
—_None
тридцать_NUMR
пятьдесят_NUMR
рублей.советовать_INFN
обратиться_INFN
гострудинспекция_NOUN
здравствовать_INFN
согласно_PREP
п.45_None
ст.346.43_None
нк_None
патентный_ADJF
применяться_INFN
розничный_ADJF
осуществлять_INFN
стационарный_ADJF
торговый_ADJF
торговый_ADJF
квадратный_ADJF
т.е_None
подтвердить_INFN
торговый_ADJF
выделить_

...

​здравствовать_INFN
советовать_INFN
ознакомиться_INFN
верховный_ADJF
29.05.2012_None
n_None
судебный_ADJF
являться_INFN
влечь_INFN
досрочный_ADJF
например_CONJ
кредитный_ADJF
обязать_INFN
возвратить_INFN
получить_INFN
наследодатель_NOUN
денежный_ADJF
уплатить_INFN
предусмотреть_INFN
заём_NOUN
предоставить_INFN
наследодатель_NOUN
личный_ADJF
семейный_ADJF
домашний_ADJF
иной_ADJF
связанный_ADJF
предпринимательский_ADJF
возвратить_INFN
досрочно_ADVB
полностью_ADVB
тридцать_NUMR
кредитный_ADJF
установленный_ADJF
короткий_ADJF
предоставить_INFN
иной_ADJF
возвратить_INFN
досрочно_ADVB
гк_None
исковый_ADJF
наследодатель_NOUN
продолжать_INFN
прерывать_INFN
пресекать_INFN
приостанавливать_INFN
предъявить_INFN
остаться_INFN
исковый_ADJF
начать_INFN
наследодатель_NOUN
наступить_INFN
исковый_ADJF
исчисляться_INFN
общий_ADJF
например_CONJ
установленный_ADJF
общий_ADJF
исковый_ADJF
предъявить_INFN
принять_INFN
—_None
наследственный_ADJF
—_None
включительно_ADVB
—_None
включительно_ADVB
исковый_ADJF


здравствуйте.в_GRND
писать_INFN
должный_ADJF
получить_INFN
регистрационный_ADJF
местанахождение_NOUN
предоставить_INFN
направить_INFN
регистрировать_INFN
оффшорный_ADJF
зарегистрировать_INFN
британский_ADJF
виргинский_ADJF
оставить_INFN
т.к_None
свой_ADJF
неоднократный_ADJF
сей_VERB
получить_INFN
вследствие_PREP
арбитражный_ADJF
невозможно_ADVB
лина—_None
направлять_INFN
добрый_ADJF
общий_ADJF
привлечь_INFN
выходной_ADJF
письменный_ADJF
согласный_NOUN
—_None
увольнять_INFN
общий_ADJF
общий_ADJF
привлечь_INFN
выходной_ADJF
сильно_ADVB
сомневаться_INFN
применимый_ADJF
привести_INFN
целиком_ADVB
понятно_ADVB
написать_INFN
требоваться_INFN
почитать_INFN
думать_INFN
поймете.тк_NOUN
выходной_ADJF
нерабочий_ADJF
праздничный_ADJF
исключительный_ADJF
выходной_ADJF
нерабочий_ADJF
праздничный_ADJF
выходной_ADJF
нерабочий_ADJF
праздничный_ADJF
запрещаться_INFN
предусмотренный_ADJF
выходной_ADJF
нерабочий_ADJF
праздничный_ADJF
производиться_INFN
письменный_ADJF
заранее_ADVB
непредвиденный_ADJF
сроч

включать_INFN
осуществлять_INFN
самостоятельно_ADVB
принудительный_ADJF
либо_CONJ
взыскателю.​таким_NOUN
пристать_INFN
вправе_PRED
забрать_INFN
реализовать_INFN
формально_ADVB
вынести_INFN
судебный_ADJF
получить_INFN
исполнительный_ADJF
далёкий_ADJF
иной_ADJF
исполнить_INFN
вынести_INFN
денежный_ADJF
невозможно_ADVB
—_None
p.s_None
детальный_ADJF
интересовать_INFN
расширить_INFN
исковый_ADJF
иной_ADJF
обратиться_INFN
—_None
получить_INFN
buffer25266_None
triada_171220/02_None
вскрыть_INFN
упаковку-прямая_NOUN
полиэдр_NOUN
цвета.ваш_NOUN
мененджер_NOUN
утверждать_INFN
чисто_ADVB
немецкий_ADJF
бафера_NOUN
уретановый_ADJF
бафера_NOUN
дезцветной_ADJF
бафера_NOUN
сделать_INFN
получить_INFN
называться_INFN
alexandr_None
filatovдобрый_ADJF
дать_INFN
необходимый_ADJF
направить_INFN
обратно_ADVB
письменный_ADJF
уплатить_INFN
суммы.ст_NOUN
26.1_None
—_None
вправе_PRED
отказаться_INFN
—_None
надлежащий_ADJF
предоставить_INFN
письменный_ADJF
вправе_PRED
отказаться_INFN
надлежащий_ADJF
возможный_AD

здравствуйте.однако_NOUN
посчитать_INFN
данный_ADJF
недостаточно_ADVB
обоснованный_ADJF
продлить_INFN
дополнительный_ADJF
данный_ADJF
отпустили.екатериназначить_INFN
дополнительный_ADJF
предоставили.ь_NOUN
вновь_ADVB
задержать_INFN
заключить_INFN
екатеринаст_NOUN
упк_NOUN
рф9_None
повторный_ADJF
уголовный_ADJF
возможно_CONJ
новый_ADJF
обосновывать_INFN
стражу.переть_INFN
этомподозревать_INFN
экономический_ADJF
заниматься_INFN
убэп_NOUN
екатерина1.1_None
применить_INFN
подозревать_INFN
обвинять_INFN
предусмотренный_ADJF
—_None
159.3_None
159.5_None
159.6_None
совершить_INFN
предпринимательский_ADJF
сообщить_INFN
следственный_ADJF
возбудить_INFN
подозревать_INFN
задержать_INFN
продолжать_INFN
служить_INFN
обычный_ADJF
проходить_INFN
срочный_ADJF
переболеть_INFN
подписал.контракт_NOUN
успел.заболеть_INFN
последний_ADJF
обнаружить_INFN
экг_None
пролкжало_NOUN
местный_ADJF
госпетала_ADJS
гдк_None
поставить_INFN
меокордит_NOUN
перевести_INFN
главный_ADJF
нде_None
исключить_INFN
поставить_INF

продать_INFN
—_None
попечительства​_None
гк_None
жилой_NOUN
принадлежащий_ADJF
подопечный_ADJF
возможно_CONJ
подопечного​_None
фз_None
иной_ADJF
приёмный_ADJF
покупать_INFN
жильё_NOUN
худой_ADJF
проживать_INFN
оформлять_INFN
потребоваться_INFN
письменный_ADJF
преимущественный_ADJF
добрый_ADJF
пояснить_INFN
дробный_ADJF
выполнять_INFN
​кто_ADVB
выступать_INFN
заключить_INFN
трудовой_ADJF
обязать_INFN
подчиняться_INFN
россии.статие_NOUN
трудовой_ADJF
трудовой_ADJF
трудовой_ADJF
трудовой_ADJF
—_None
обязываться_INFN
предоставить_INFN
обусловить_INFN
трудовой_ADJF
обеспечить_INFN
предусмотренный_ADJF
трудовой_ADJF
иной_ADJF
нормативный_ADJF
правовой_ADJF
содержимый_ADJF
трудовой_ADJF
коллективный_ADJF
локальный_ADJF
нормативный_ADJF
данные_PRTF
своевременно_ADVB
полный_ADJF
выплачивать_INFN
заработный_ADJF
обязываться_INFN
лично_ADVB
выполнять_INFN
определённый_ADJF
трудовой_ADJF
соблюдать_INFN
внутренний_ADJF
трудовой_ADJF
действующий_ADJF
данный_ADJF
трудовой_ADJF
являться_INFN
работник.

In [29]:
import random
def w2v_search(question, base_dict):
    scores = []
#     question  = random.choice(questions)
    print('question - ', question)
    prepr = preprocessing(question, del_stopwords=True, del_digit=True)
#     print(prepr)
    get_wv = get_w2v_vectors(prepr, question)
#     print('aaa',get_wv.get(question))
#     print(type(val))
    for key, value in base_dict.items():
#         print(value)
        for key1, value1 in get_wv.items():
            
#         print(get_wv[key])
            try:
                sim = similarity(value, value1)
                scores.append((key, sim))
            except ValueError:
                print('string')
#     scores = sorted(scores, key=lambda x: x[1], reverse=True)
#     return scores[:5]
    return scores
    

# search = w2v_search(questions, save_wv)
# search

Doc2Vec

In [13]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec.load('D:/Doc2Vec_100s_1000e')

In [25]:
def preprocessing(input_text, del_digit=True):
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
#     lemmas_ = ' '.join(lemmas_arr)
    return lemmas_arr


def get_d2v_vectors(lemmas_arr, text):
    """Получает вектор документа"""
    new_vec = model.infer_vector(lemmas_arr)
    text = text.strip()
    text = text.replace('\xa0', '')
    text = text.replace('\u200b', '')
    dict_ = {text :  new_vec}
    return dict_


def save_d2v_base(dict_, base_dict):
    """Индексирует всю базу для поиска через doc2vec"""
    base_dict.update(dict_)
    return base_dict

In [18]:
base_dict = {}
for answer in answers:
    prepr = preprocessing(answer, del_digit=True)
    print(1)
    get_dv = get_d2v_vectors(prepr, answer)
    print(2)
    save_dv = save_d2v_base(get_dv, base_dict)
    print(3)

1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2


2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3
1
2
3


In [21]:
def d2v_search(question, save_dv):
    scores = []
#     question = random.choice(questions)
    print('question - ', question)
    prepr = preprocessing(question, del_digit=True)
    print('3')
    get_dv = get_d2v_vectors(prepr, question)
    for key, value in save_dv.items():
        for key1, value1 in get_dv.items():
            try:
                sim = similarity(value, value1)
                scores.append((key, sim))
            except ValueError:
                print('string')
#     scores = sorted(scores, key=lambda x: x[1], reverse=True)
#     return scores[:5]
    return scores
    



In [20]:
import random
search = d2v_search(questions, save_dv)
search

question -  
Я живу на третьем этаже, на первом этаже под моей квартирой находится небольшой продуктовый магазин, на пятом этаже живет пенсионерка, у которой на балконе сделана самодельная крыша со скатом. На этой крыше «выросла» огромная сосулька. Старшая по дому решила эту сосульку убрать, чтобы она случайно не упала на голову покупателям магазина. Она пригласила жильца из другого подъезда, чтобы он сбил сосульку. На моем балконе есть бельевые веревки на улице (сам балкон застеклен). Эти люди встретились мне в подъезде, когда шли наверх. Пошутили «ни о чем» про сосульку не сказали. Я ушла из дома, а когда вечером вернулась, обнаружила, что трубы с веревками почти полностью согнуты. Закреплены они под обшивкой балкона, просто так к ним не подберешься. Кто теперь должен мне это починить? Я хочу написать на них заявление, но куда? У нас управляющая компания.

3


c:\users\maria tsareva\appdata\local\programs\python\python36-32\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Я живу на третьем этаже, на первом этаже под моей квартирой находится небольшой продуктовый магазин, на пятом этаже живет пенсионерка, у которой на балконе сделана самодельная крыша со скатом. На этой крыше «выросла» огромная сосулька. Старшая по дому решила эту сосульку убрать, чтобы она случайно не упала на голову покупателям магазина. Она пригласила жильца из другого подъезда, чтобы он сбил сосульку. На моем балконе есть бельевые веревки на улице (сам балкон застеклен). Эти люди встретились мне в подъезде, когда шли наверх. Пошутили «ни о чем» про сосульку не сказали. Я ушла из дома, а когда вечером вернулась, обнаружила, что трубы с веревками почти полностью согнуты. Закреплены они под обшивкой балкона, просто так к ним не подберешься. Кто теперь должен мне это починить? Я хочу написать на них заявление, но куда? У нас управляющая компания. Здравствуйте, Ольга! Для начала напишите претензию управляющей компании, сделайте приписку, что в случае неудовлетворения Вашей претензии ос

OKAPI BM25

In [24]:
from math import log
from collections import Counter

In [25]:
def preprocessing(input_text, del_stopwords=True, del_digit=True):
    russian_stopwords = set(get_stop_words('russian'))
    words = [x.lower().strip(string.punctuation + '»«–…') for x in word_tokenize(input_text)]
    lemmas = [morph.parse(x)[0].normal_form for x in words if x]
    lemmas_arr = []
    for lemma in lemmas:
        if del_stopwords:
            if lemma in russian_stopwords:
                continue
        if del_digit:
            if lemma.isdigit():
                continue
        lemmas_arr.append(lemma)
    return lemmas_arr

In [26]:
answers_ = []
for answer in answers:
    prepr = preprocessing(answer, del_stopwords=True, del_digit=True)
    answers_.append(prepr)

In [27]:
sum_len = 0
for answer in answers:
    sum_len += len(answer)
av_len = float(len(answers) / sum_len)
print(av_len)

0.001046687242440445


In [49]:
def freq(question, answers_, answers):
    question = preprocessing(question, del_stopwords=True, del_digit=True)
    counts = {}
    for answer_, answer in zip(answers_, answers):
        answer = answer.replace('\xa0', '')
        answer = answer.replace('\u200b', '')
        # text_ = preprocessing(text, del_stopwords=True, del_digit=True)
        text_len = len(answers_)
        for ques in question:
            count = Counter(answer_)[ques]
            counts[answer] = [count, text_len]
    # n = 0
    # for key, value in counts.items():
    #     dl = value[1]
    #     qf = value[0]
#     print(counts)
    return counts

In [44]:
k1 = 2.0
b = 0.75


def score_BM25(qf, dl, avgdl, k1, b, N, n, counts) -> float:
    score = log((N - n + 0.5) / (n + 0.5)) * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))
    return score

In [60]:
def fin_ok(question, answers, N, av_len):
    q = freq(question, answers_, answers)
    n = 0
    for key, value in q.items():
        # print(key, value)
        if value[0] != 0:
            n += 1
        dl = value[1]
        qf = value[0]
        rev_score = score_BM25(qf, dl, av_len, k1, b, N, n, q)
#         print(rev_score)
        value.append(rev_score)
#         print(value)
        value.remove(value[0])
        value.remove(value[0])
#         print(value)
    scores = sorted(q, key=q.get, reverse=True)
    return scores[:5]

In [63]:
import random
question  = random.choice(questions)
print('question - ', question)
N = len(answers)
fin = fin_ok(question, answers, N, av_len)
fin

question -  
здравствуйте! у нас вот такая ситуация:  10 января у нас заканчивается срок временного убежища, для подачи на гражданства РФ нужно чтоб этому документу было 6 месяцев. третий раз нам его продлить не захотели, потому что есть РВП, сказали забирайте укр паспорта и ставьте РВП туда и делайте перевод... у мужа просрочен паспорт, вклеена фото в 25 лет, сейчас ему 27, получается просрочка 2 года. Рвп ему в паспорт не переставили, потому что нет документа удостоверяющего личность, сказали ехать в посольство делать справку о выезде на границу и продлевать паспорт там, на укр сторону ему ехать нельзя.  у меня укр паспорт действителен до мая
, мне в мае 25 лет, РВП стоит в укр паспорте. как быть??? соотечественники есть до 8.08.2018г.... РВП и у меня и у мужа до 2019 года, но у мужа оно не где не проставлено .... можно ли заново подать документы на временное убежище??



['Добрый день! Право на постоянное проживание на территории РФ иностранным гражданам и лицам без гражданства даёт Вид на жительство (ВНЖ). В общем порядке нужно сначала получить РВП, прожить в РФ по РВП не менее года и только затем можно обратиться за получением ВНЖ. Вид на жительство выдается иностранному гражданину на пять лет и может быть продлён неограниченное количество раз. Но существует упрощённый режим получения ВНЖ, без необходимости получения РВП. Итак, получить ВНЖ, упрощённо, могут: 1) граждане р. Беларусь на основании двустороннего международного соглашения между РФ и р. Беларусь от 24.01.2006 г. 2) лица, признаные в установленном порядке носителями русского языка. Особенность данного основания заключается в следующем: а) срок действия ВНЖ составляет 3 года ( в обычном пордке — это 5 лет);  б) заявитель имеет право обращаться с заявлением о выдаче вида на жительство только в тот территориальный орган по делам миграции МВД РФ, решением комиссии которого он был признан носит